In [1]:
import anndata
import os
import requests

save_path = "data/example_sce.h5ad"
if not os.path.exists(save_path):
    response = requests.get("https://go.wisc.edu/69435h")
    with open(save_path, "wb") as f:
        f.write(response.content)

example_sce = anndata.read_h5ad(save_path)
example_sce

AnnData object with n_obs × n_vars = 2087 × 100
    obs: 'clusters_coarse', 'clusters', 'S_score', 'G2M_score', 'cell_type', 'sizeFactor', 'pseudotime'
    var: 'highly_variable_genes'
    uns: 'X_name', 'clusters_coarse_colors', 'clusters_colors', 'day_colors', 'neighbors', 'pca'
    obsm: 'PCA', 'UMAP', 'X_pca', 'X_umap'
    layers: 'counts', 'cpm', 'logcounts', 'spliced', 'unspliced'
    obsp: 'connectivities', 'distances'

In [2]:
from scdesigner.experimental.estimators import negative_binomial_regression
from scdesigner.experimental.transform import nullify, amplify, data_frame_mask

params = negative_binomial_regression(example_sce, "~ bs(pseudotime, degree=2)", epochs=10)
outcomes = example_sce.var_names[:4]
mask = data_frame_mask(params["coefficient"], ["pseudotime"], outcomes)
null_params = nullify(params, "coefficient", mask)
null_params["coefficient"]

,Pyy,Iapp,Chgb,Rbp4,Spp1,Chga,Cck,Ins1,Nnat,Ins2,...,Nkx6-1,Fxyd3,Hn1,Smarcd2,Pdia6,Ffar2,Hes6,Serpinh1,Npy,1110012L19Rik
Intercept,1.733727,1.689458,1.651231,1.555795,2.025110,1.444613,1.584103,1.572423,1.389907,1.395772,...,0.622493,-0.016563,1.325979,0.732366,1.021546,-0.071549,0.967869,1.721433,-1.769911,-0.324305
"bs(pseudotime, degree=2)[1]",0.000000,0.000000,0.000000,0.000000,1.550395,1.539664,1.716174,0.460826,0.275353,-1.156779,...,0.849134,1.638570,1.285707,2.082242,-1.730549,2.030675,1.460706,-0.683348,-2.834935,2.759499
"bs(pseudotime, degree=2)[2]",0.000000,0.000000,0.000000,0.000000,-3.783759,1.599532,1.261496,1.685368,1.671482,1.665799,...,0.968832,0.368252,-1.606683,-3.385910,1.537826,0.048395,-1.933420,-3.407624,2.610776,-1.205206


In [3]:
from scdesigner.experimental.estimators import negative_binomial_copula

params = negative_binomial_copula(example_sce, "~ bs(pseudotime, degree=2)", epochs=10)
mask = data_frame_mask(params["covariance"], outcomes)
null_params = nullify(params, "covariance", mask)
null_params["covariance"]

,Pyy,Iapp,Chgb,Rbp4,Spp1,Chga,Cck,Ins1,Nnat,Ins2,...,Nkx6-1,Fxyd3,Hn1,Smarcd2,Pdia6,Ffar2,Hes6,Serpinh1,Npy,1110012L19Rik
Pyy,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Iapp,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Chgb,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Rbp4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Spp1,0.393770,0.799731,-0.289737,0.295339,2.397488,-0.242628,-0.899595,0.567017,0.414025,0.321058,...,-0.088210,-0.073994,-0.002064,-0.197042,0.415797,-0.022235,-0.102545,0.037854,-0.101790,-0.020583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ffar2,0.005455,0.017246,0.236328,0.000351,-0.022235,0.135657,-0.001113,0.021619,0.018214,0.036313,...,0.020421,0.090492,-0.045606,0.017541,0.009610,0.842480,-0.019808,-0.020442,-0.021746,0.048316
Hes6,0.307701,0.639868,-0.103832,0.168385,-0.102545,-0.073484,-0.037888,0.366841,0.378034,0.348445,...,0.159745,-0.019175,0.405133,0.413313,0.184725,-0.019808,1.161238,0.351598,-0.034006,0.080193
Serpinh1,0.459500,0.652974,-0.049493,0.291291,0.037854,0.013958,-0.167526,0.386435,0.378422,0.374376,...,0.079299,-0.009366,0.287636,0.343232,0.160990,-0.020442,0.351598,1.064454,-0.040935,0.085780
Npy,-0.421262,-0.243200,-0.153367,-0.306399,-0.101790,-0.113568,0.232231,0.079748,-0.021376,0.048970,...,0.085588,-0.020925,0.024532,-0.052170,0.058338,-0.021746,-0.034006,-0.040935,0.432047,-0.022467


In [4]:
mask = data_frame_mask(params["coefficient"], ["pseudotime"], outcomes)
null_params = amplify(params, "coefficient", mask, factor=2)
null_params["coefficient"] / params["coefficient"]

,Pyy,Iapp,Chgb,Rbp4,Spp1,Chga,Cck,Ins1,Nnat,Ins2,...,Nkx6-1,Fxyd3,Hn1,Smarcd2,Pdia6,Ffar2,Hes6,Serpinh1,Npy,1110012L19Rik
Intercept,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"bs(pseudotime, degree=2)[1]",2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"bs(pseudotime, degree=2)[2]",2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
